# Classe FlowBase

Classe que implementa o fluxo Padrão dos fluxos.

In [2]:
from pyspark.sql.functions import col, concat, regexp_replace, max
from pyspark.sql.types import IntegerType
from datetime import datetime
import json
import hashlib

class FlowBase():
    def __init__(self, name):
        self.name = name
        
        df = self.run()
        self.writeFlowResult(df)
        
    def run(self):
        return
    
    @staticmethod
    def loadUsedDatasets(dataset):
        id_dataset, name, bucket_name, path, delimiter, extension, created_at = get_configs_dataset(dataset.lower())
        datasetAux = DatasetBase(bucket_name, path, delimiter, extension)
        complete_path = f'/mnt/{dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")}/raw/{name.lower()}/{datetime.now().strftime("%Y/%m/05/")}'
        df = datasetAux.loadDatabase(bucket_name, complete_path)
        return df
      
    @staticmethod
    def checkDuplicates(df):
        if df.count() > df.dropDuplicates(df.schema.names).count():
            return True
        else:
            return False
          
    @staticmethod
    def checkColumnsNull(df, columns):
        for column in columns:
            if df.where(col(column).isNull()).count() > 0:
                return True
        return False
    
    def writeFlowResult(self, df):
        bucket_write = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
        complete_path = f'/mnt/{bucket_write}/curated/enriched/{self.name}/{datetime.now().strftime("%Y/%m/%d/")}'
        DatasetBase.writeDataset(df, bucket_write, complete_path)
        
    import hashlib
    @staticmethod
    def encrypt_value(value):
        sha_value = hashlib.sha256(value.encode()).hexdigest()
        return sha_value
      
    @staticmethod
    def parse_json(array_str):
        json_obj = json.loads(array_str)
        for item in json_obj:
            yield (          item["name"],item["addition"],item["discount"],item["quantity"],item["sequence"],item["unitPrice"],item["externalId"],item["totalValue"],item["customerNote"],item["garnishItems" ],item["integrationId"],item["totalAddition"],item["totalDiscount"])